# **Comparative Analysis of Tree Ensemble Models for EEG Event Detection**
This notebook is a comparative analysis of tree ensemble models for EEG event detection. I will be evaluating the performance of the following models:
1. Random Forest
2. Gradient Boosting
3. XGBoost

I will be using a simple pipeline to preprocess the data and train the models. The pipeline will consist of the following steps:
1. Feature extraction from the EEG channels Using Common Spatial Patterns (CSP).
2. Multi label classification using the tree ensemble models mentioned above.

I will be performing a grid search for hyper-parameter tuning of each model, and then will also be using five fold cross validation to hopefully protect against overfitting.
I will compare the performance of the models using the following metrics:
1. Precision
2. Recall
3. F1 Score
4. Confusion Matrix

I am using data from this [Grasp-and-Lift EEG Detection](https://www.kaggle.com/c/grasp-and-lift-eeg-detection/) competition. The data consists of EEG recordings from 32 channels for 10 subjects. The data is divided into 45 trials for each subject. Each trial consists of 6 events. The goal is to detect the start and end of each event in the EEG recordings. This competition is from 2015, but it serves as an excellent data set for benchmarking EEG event detection algorithms.This will also allow me to test the effectiveness of these models against all the submissions to this competition.

I was inspired by the work being done at Stanford University on [NOIR: Neural Signal Operated Intelligent Robots](https://noir-corl.github.io/). They are using CSP And quadratic discriminate analysis(QDA) to process EEG signals, and have had amazing results with study participants being able to control robots. The data set I am working with here however is structured as a multi label classification problem, so I opted To compare the efficacy of tree ensembles in place of QDA. I believe that EEG signals have the potential to revolutionize human-computer interaction, especially for people with physical disabilities like myself. I hope that this notebook will inspire others to work on EEG event detection and other EEG applications.

## **Install The Libraries**
First we install install all necessary Python libraries. Check the [README.md](../README.md) file for more info on how to do this.

## **Kaggle Environment Setup**
You will need to upload your *kaggle.json*, set the permissions so the file can be read.

In [ ]:
!chmod 600 ../kaggle.json

Then we set the Kaggle configuration directory to our current working directory, as an environment variable.

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '../'

Now we can download the data from the competition page, 

In [ ]:
if not os.path.exists('../data/kaggle-eeg'):
    os.makedirs('../data/kaggle-eeg')
    !kaggle competitions download grasp-and-lift-eeg-detection -p ../data/kaggle-eeg/ -f train.zip
    !unzip ../data/kaggle-eeg/train.zip -d ../data/kaggle-eeg

## **Imports**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mne.decoding import CSP
from xgboost import XGBClassifier, plot_importance, cv
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, hamming_loss, jaccard_score, multilabel_confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
import wandb
pd.set_option('display.max_columns', None)


## **Data Analysis**
First we load some of the training data and check the first few rows.

In [ ]:
data_path = '../data/kaggle-eeg/train'
features = pd.read_csv(f'{data_path}/subj1_series1_data.csv')
labels = pd.read_csv(f'{data_path}/subj1_series1_events.csv')
features = features.drop(columns=['id'])
labels = labels.drop(columns=['id'])

display(features.info(), features.describe(), features.head(), labels.info(), labels.describe(), labels.head())

In [ ]:
def load_series_data(subject, series):
    features = pd.read_csv(f'{data_path}/subj{subject}_series{series}_data.csv')
    labels = pd.read_csv(f'{data_path}/subj{subject}_series{series}_events.csv')
    return features, labels

features, labels = load_series_data(1, 1)

In [ ]:
corr = features.drop(columns=['id']).corr()
plt.figure(figsize=(30, 20))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.show()

## **Modeling**

### **Data Preprocessing**

In [ ]:
def merge_labels(features, labels):
    data = features.copy()
    data = data.merge(labels, on='id')
    data.drop(columns=['id'], inplace=True)
    return data

def get_training_batch(series):
    features, labels = load_series_data(1, series)
    data = merge_labels(features, labels)
    for i in range(2, 13):
        features, labels = load_series_data(i, series)
        data = pd.concat([data, merge_labels(features, labels)])
    return data

In [ ]:
train_df = get_training_batch(1)
train_df.shape, train_df.value_counts()

In [ ]:
labels = train_df[['HandStart', 'FirstDigitTouch', 'BothStartLoadPhase', 'LiftOff', 'Replace', 'BothReleased']]
unique_label_combinations, counts = np.unique(labels, axis=0, return_counts=True)

plt.figure(figsize=(30, 10))
sns.barplot(x=range(len(unique_label_combinations)), y=counts, order=np.argsort(counts)[::-1])
plt.title('Label Frequency')
plt.xticks(range(len(unique_label_combinations)), unique_label_combinations, rotation=45)
plt.xlabel('Label Combination')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# reshape data for CSP
def reshape_data(data):
    X = data.drop(columns=['HandStart', 'FirstDigitTouch', 'BothStartLoadPhase', 'LiftOff', 'Replace', 'BothReleased']).values
    y = data[['HandStart', 'FirstDigitTouch', 'BothStartLoadPhase', 'LiftOff', 'Replace', 'BothReleased']].values
    y = y.astype(np.float64)
    X = np.expand_dims(X, axis=2)
    X = X.astype(np.float64)
    return X, y

X_train, y_train = reshape_data(train_df)
X_train = X_train.astype('float64').copy()
X_train.shape, y_train.shape, X_train.dtype, y_train.dtype

### **Model Building**

In [ ]:
def build_model(pipeline, param_grid, X_train, y_train, outer_cv=5, inner_cv=5, scoring='jaccard'):
    outer_cv = KFold(n_splits=outer_cv, shuffle=True, random_state=42)
    scores = []
    best_estimators = [] # Store a list of best estimators for each label
    best_scores = [] # Store a list of best scores for each label

    for train_index, test_index in outer_cv.split(X_train):
        X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
        
        # Initialize list to store best estimators and scores for each label in this outer fold
        fold_best_estimators = []
        fold_best_scores = []

        # Iterate over each label (column) in the multi-label target matrix
        for label_idx in range(y_train_fold.shape[1]):
            y_train_label = y_train_fold[:, label_idx]
            y_test_label = y_test_fold[:, label_idx]

            inner_cv = KFold(n_splits=inner_cv, shuffle=True, random_state=42)
            grid_search = GridSearchCV(pipeline, param_grid, cv=inner_cv, verbose=1, scoring='accuracy', n_jobs=8) # You may change scoring to other suitable multi-label metrics
            grid_search.fit(X_train_fold, y_train_label)

            y_pred = grid_search.predict(X_test_fold)
            
            # Example: Calculate multiple metrics
            accuracy = accuracy_score(y_test_label, y_pred)
            jaccard = jaccard_score(y_test_label, y_pred, average='samples')  
            hamming = hamming_loss(y_test_label, y_pred)

            scores.append({'accuracy': accuracy, 'jaccard': jaccard, 'hamming': hamming}) 
            fold_best_estimators.append(grid_search.best_estimator_)
            fold_best_scores.append(jaccard)  

        best_estimators.append(fold_best_estimators)
        best_scores.append(fold_best_scores)
        

    best_label_indices = []
    for scores_per_fold in best_scores:
        best_label_indices.append(np.argmax(scores_per_fold))

    best_estimators_final = []
    for i in range(len(best_estimators)):  # Outer fold
        best_estimators_final.append(best_estimators[i][best_label_indices[i]])

    best_estimator = best_estimators_final[np.argmax([score[scoring] for score in scores])]
    return best_estimator, scores


#### Random Forest Model

In [ ]:
clf = RandomForestClassifier()
rf_pipeline = Pipeline([('CSP', CSP(n_components=4)),
                     ('RF', clf)])

param_grid = {
    'CSP__n_components': [2, 4, 6],
    'RF__n_estimators': [50, 100, 200],
    'RF__max_depth': [3, 5, 7]
}

best_rf_pipeline, scores= build_model(rf_pipeline, param_grid, X_train, y_train)
print("Grid search scores: ", scores)
rf_cv = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(best_rf_pipeline, X_train, y_train, cv=rf_cv, scoring='accuracy')
print("Cross-validation scores: ", scores)
print("Best RF parameters: ", best_rf_pipeline.get_params())

#### Gradient Boosting Model

In [ ]:
clf = GradientBoostingClassifier()
gb_pipeline = Pipeline([('CSP', CSP(n_components=4)),
                        ('GB', clf)])

param_grid = {
    'CSP__n_components': [2, 4, 6],
    'GB__n_estimators': [50, 100, 200],
    'GB__max_depth': [3, 5, 7]
}

best_gb_pipeline, scores = build_model(gb_pipeline, param_grid, X_train, y_train)
print("Grid search scores: ", scores)
gb_cv = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(best_gb_pipeline, X_train, y_train, cv=gb_cv, scoring='accuracy')
print("Cross-validation scores: ", scores)
print("Best GB parameters: ", best_gb_pipeline.get_params())

#### XGBoost Model

In [ ]:
clf = MultiOutputClassifier(XGBClassifier(), n_jobs=8)
xgb_pipeline = Pipeline([('CSP', CSP(n_components=4)), 
                        ('XGB', clf)])

param_grid = {
    'CSP__n_components': [2, 4, 6],
    'XGB__estimator__n_estimators': [50, 100, 200],
    'XGB__estimator__max_depth': [3, 5, 7],
    'XGB__estimator__learning_rate': [0.01, 0.1, 0.3],
    'XGB__estimator__gamma': [0, 0.1, 0.3],
    'XGB__estimator__subsample': [0.5, 0.7, 1],
    'XGB__estimator__colsample_bytree': [0.5, 0.7, 1],
    'XGB__estimator__reg_alpha': [0, 0.1, 0.3],
    'XGB__estimator__reg_lambda': [0, 0.1, 0.3],
    'XGB__estimator__eta': [0.01, 0.1, 0.3],
}

best_xgb_pipeline, scores = build_model(xgb_pipeline, param_grid, X_train, y_train)
print("Grid search scores: ", scores)
xgb_cv = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(best_xgb_pipeline, X_train, y_train, cv=xgb_cv, scoring='accuracy')
print("Cross-validation scores: ", scores)
print("Best XGB parameters: ", best_xgb_pipeline.get_params())

### **Training Loop**

In [ ]:
def training_loop(model):
    total_test_x = pd.DataFrame()
    total_test_y = pd.DataFrame()
    for series in range(1, 9):
        train_df = get_training_batch(series)
        X_train, y_train = reshape_data(train_df)
        train_x, test_x, train_y, test_y = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
        model.fit(train_x, train_y)
        total_test_x = pd.concat([total_test_x, test_x])
        total_test_y = pd.concat([total_test_y, test_y])
        score = accuracy_score(total_test_y, model.predict(total_test_x))
        print(f"Series {series} score: {score}")
    y_pred = model.predict(total_test_x)
    accuracy = accuracy_score(total_test_y, y_pred)
    report = classification_report(total_test_y, y_pred)
    return accuracy, report, multilabel_confusion_matrix(total_test_y, y_pred)

### **Training and Evaluation**

#### Random Forest Model

In [ ]:
rf_accuracy, rf_report, rf_confusion_matrix = training_loop(best_rf_pipeline)
print(f"Random Forest accuracy: {rf_accuracy}")
print(rf_report)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(30, 10))

ConfusionMatrixDisplay(rf_confusion_matrix[0]).plot(ax=ax[0])
estimator = best_rf_pipeline.named_steps['RF']
importances = estimator.feature_importances_
sns.barplot(x=range(len(importances)), y=importances, ax=ax[1])

plt.show()

#### Gradient Boosting Model

In [ ]:
gb_accuracy, gb_report, gb_confusion_matrix = training_loop(best_gb_pipeline)
print(f"Gradient Boosting accuracy: {gb_accuracy}")
print(gb_report)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(30, 10))

ConfusionMatrixDisplay(gb_confusion_matrix[0]).plot(ax=ax[0])
estimator = best_gb_pipeline.named_steps['GB']
importances = estimator.feature_importances_
sns.barplot(x=range(len(importances)), y=importances, ax=ax[1])

plt.show()

#### XGBoost Model

In [ ]:
xgb_accuracy, xgb_report, xgb_confusion_matrix = training_loop(best_xgb_pipeline)
print(f"XGBoost accuracy: {xgb_accuracy}")
print(xgb_report)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(30, 10))

ConfusionMatrixDisplay(xgb_confusion_matrix[0]).plot(ax=ax[0])
estimator = best_xgb_pipeline.named_steps['XGB']
plot_importance(estimator, ax=ax[1])

plt.show()

## **Submission**
We're gonna download the testing data now from the Kaggle competition and unzip into the data directory.

In [ ]:
!kaggle competitions download grasp-and-lift-eeg-detection -f test.zip

In [ ]:
!unzip ../data/kaggle-eeg/test.zip -d ../data/kaggle-eeg

Here we load the sample submission from the Kaggle competition. This gives us a pre-made dataframe and we just need to update column values with predictions from our model. 

In [ ]:
!kaggle competitions download grasp-and-lift-eeg-detection -f sample_submission.csv.zip

In [ ]:
!unzip ../data/kaggle-eeg/sample_submission.csv.zip -d ../data/kaggle-eeg

In [ ]:
sub = pd.read_csv('../data/kaggle-eeg/sample_submission.csv')

In [ ]:
sub.head()

Here we create a dataframe in the same shape as the example submission on the competition page.

In [ ]:
path = '../data/kaggle-eeg/test'

def get_merged_tests():
  tests = None
  for sj in range(1, 13):
    for sr in range(9, 11):
      c_tests = pd.read_csv(f'{path}/subj{sj}_series{sr}_data.csv')
      tests = c_tests if tests is None else pd.concat([tests, c_tests])
  return tests

In [ ]:
tests = get_merged_tests()

In [ ]:
tests = tests.drop(columns=['id'])
tests.head()

In [ ]:
def get_predictions(model):
    data = tests.values
    data = np.expand_dims(data, axis=2)
    data = data.astype('float64')
    return model.predict(data)

In [ ]:
test_predictions_rf = get_predictions(best_rf_pipeline)
test_predictions_gb = get_predictions(best_gb_pipeline)
test_predictions_xgb = get_predictions(best_xgb_pipeline)

In [ ]:
def create_submission(predictions, message):
    sub.iloc[:, 1:] = predictions
    sub.to_csv('../data/kaggl-eeg/submission.csv', index=False)
    submit_cmd = f'!kaggle competitions submit grasp-and-lift-eeg-detection -f ../data/kaggle-eeg/submission.csv -m "{message}"'
    !{submit_cmd}